### Baseline Model

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models, layers, optimizers

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random

In [2]:
main = os.listdir("../raw_data/wikiart")

genres = []
for genre in main:
    if genre[0] != ".":
        genres.append(genre)

data = sorted(genres)

In [3]:
poc_list = ["Realism", "Impressionism"]

In [4]:
from PIL import Image
import glob, os

paintings_list = []
size = 128, 128

for index, genre in enumerate(poc_list):
    print(f"the genre is {genre}")
    g = os.listdir("../raw_data/wikiart/" + str(genre))
    
    for i, painting in enumerate(g):
        with Image.open("../raw_data/wikiart/" + genre + "/" +painting) as im:
            img_resized = im.resize(size)
            image_array = np.array(img_resized)
            #print(image_array.shape)
            #paintings_list.append(image_array)
            string = g[i][:-4]
            string = string.replace("-"," ")
            string = string.split("_")
            string.insert(0, genre)
            string.insert(3, image_array)
            paintings_list.append(string)
    
            if i % 100 == 0:
                print(i+1)

the genre is Realism
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901
10001
10101
10201
10301
10401
10501
10601
10701
the genre is Impressionism
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
850

In [5]:
df = pd.DataFrame(paintings_list)
df

,0,1,2,3,4,5
0,Realism,konstantin somov,portrait of louise morgan,"[[[66, 49, 43], [53, 41, 40], [60, 45, 37], [7...",None,None
1,Realism,fyodor vasilyev,abandoned mill 2 1873,"[[[244, 244, 242], [243, 244, 242], [240, 246,...",None,None
2,Realism,john singer sargent,mrs augustus allusen osma mary dorothy stanley...,"[[[55, 52, 63], [52, 49, 60], [50, 47, 58], [5...",None,None
3,Realism,boris kustodiev,portrait of julia petrovna greek 1901,"[[[167, 121, 98], [146, 117, 95], [133, 107, 8...",None,None
4,Realism,henri fantin latour,still life vase of hydrangeas and ranunculus 1866,"[[[173, 148, 107], [175, 151, 109], [179, 155,...",None,None
...,...,...,...,...,...,...
23788,Impressionism,claude monet,white azaleas in a pot,"[[[210, 231, 236], [201, 230, 237], [200, 232,...",None,None
23789,Impressionism,winston churchill,interior at breccles(1),"[[[245, 237, 245], [231, 220, 209], [228, 222,...",None,None
23790,Impressionism,childe hassam,the children,"[[[127, 106, 82], [122, 87, 51], [123, 84, 45]...",None,None
23791,Impressionism,john singer sargent,unloading boats in venice 1904,"[[[255, 243, 247], [254, 242, 246], [254, 242,...",None,None


In [6]:
def load_model():
    
    # $CHALLENGIFY_BEGIN
    
    model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
    
    # $CHALLENGIFY_END
    
    return model

In [7]:
def set_nontrainable_layers(model):
    
    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model

In [8]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model


In [9]:
def build_model():
    # $CHALLENGIFY_BEGIN    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

In [10]:
def tolist(x):
    return x.tolist()

In [1]:
df

NameError: name 'df' is not defined

In [11]:
X = df[3]

In [12]:
X = np.stack(X.values)

In [43]:
#turn genre in binary
def genre_to_binary(x):
    if x == "Realism":
        return 0
    if x == "Impressionism":
        return 1

In [51]:
y = df[0].apply(genre_to_binary)

In [52]:
y

0        0
1        0
2        0
3        0
4        0
        ..
23788    1
23789    1
23790    1
23791    1
23792    1
Name: 0, Length: 23793, dtype: int64

In [53]:
y = np.stack(y)

In [54]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [57]:
X_train.shape

(12491, 128, 128, 3)

In [58]:
y_train.shape

(12491,)

In [59]:
y_train

array([1, 0, 1, ..., 1, 1, 1])

In [60]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])

Epoch 1/50
781/781 [==============================] - 1568s 2s/step - loss: 1.6101 - accuracy: 0.6876 - val_loss: 0.6839 - val_accuracy: 0.7133
Epoch 2/50
781/781 [==============================] - ETA: 0s - loss: 0.2548 - accuracy: 0.9023

KeyboardInterrupt: 

In [61]:
df = pd.read_csv("../smArt/data/binary_train_20k.csv")

In [62]:
df

,Unnamed: 0,genre,artist,title,image
0,0,Realism,abraham manievich,flowers vase on a hamper,[[[141 147 126]\n [142 150 123]\n [147 157 1...
1,1,Realism,adolf hitler,alter werderthor wien,[[[169 136 105]\n [167 134 103]\n [164 131 1...
2,2,Realism,adolf hitler,castle battlements,[[[186 182 157]\n [186 182 157]\n [184 179 1...
3,3,Realism,adolf hitler,flowers with roots,[[[220 213 203]\n [202 195 185]\n [209 200 1...
4,4,Realism,adolf hitler,informal dining room long view,[[[153 116 89]\n [160 129 128]\n [175 158 1...
...,...,...,...,...,...
19995,19995,Impressionism,zinaida serebriakova,veranda spring,[[[193 195 191]\n [196 186 178]\n [192 182 1...
19996,19996,Impressionism,zinaida serebriakova,window 1910,[[[ 92 76 77]\n [ 97 81 82]\n [ 93 77 ...
19997,19997,Impressionism,zinaida serebriakova,winter in the royal village upper bath 1912,[[[205 193 179]\n [203 191 177]\n [199 187 1...
19998,19998,Impressionism,zinaida serebriakova,winter landscape neskuchnoye 1910,[[[192 178 152]\n [192 180 156]\n [193 183 1...
